# Machine Learning for Finance Freestyle

In this lab you'll be given the opportunity to apply everything you have learned to build a trading strategy for SP500 stocks. First, let's introduce the dataset you'll be using.

## The Data

Use BigQuery's magic function to pull data as follows:

    Dataset Name: ml4f
    Table Name: percent_change_sp500

The following query will pull 10 rows of data from the table:

In [1]:
%%bigquery df
SELECT 
    *
FROM
    `cloud-training-prod-bucket.ml4f.percent_change_sp500`
LIMIT
    10

Downloading: 100%|██████████| 10/10 [00:01<00:00,  7.38rows/s]


In [2]:
df.head()

,symbol,Date,Open,Close,tomorrow_close,tomo_close_m_close,close_MIN_prior_5_days,close_MIN_prior_20_days,close_MIN_prior_260_days,close_MAX_prior_5_days,...,close_STDDEV_prior_20_days,close_STDDEV_prior_260_days,close_values_prior_260,days_on_market,scaled_change,s_p_scaled_change,normalized_change,company,industry,direction
0,CAM,2001-11-19,34.10,34.79,36.06,1.27,0.942800,0.942800,0.837884,1.167864,...,0.069786,0.301204,"[59.25, 58.25, 57.69, 56.69, 56.5, 58.0, 56.44...",1603,0.036505,-0.007298,0.043802,Cameron International Corp,Energy,UP
1,AMAT,1985-10-02,20.00,20.00,19.12,-0.88,0.950000,0.950000,0.950000,0.975000,...,0.065214,0.178471,"[31.0, 30.0, 29.12, 29.12, 29.12, 29.12, 27.75...",271,-0.044000,0.001630,-0.045630,Applied Materials Inc,Information Technology,DOWN
2,BEN,1999-11-05,33.31,33.06,31.56,-1.50,0.969752,0.828494,0.820024,1.056564,...,0.065775,0.136138,"[33.15, 32.22, 31.85, 34.58, 37.49, 36.19, 36....",3831,-0.045372,0.004948,-0.050320,Franklin Resources Inc,Financials,DOWN
3,AIG,2003-07-17,59.79,59.80,60.02,0.22,0.933946,0.922742,0.743645,1.000000,...,0.022495,0.088380,"[67.74, 67.7, 65.55, 62.4, 63.51, 63.5, 63.33,...",4759,0.003679,0.011806,-0.008127,American Intl Group Inc,Financials,STAY
4,CMA,2006-03-02,56.96,56.86,56.92,0.06,1.006155,0.962891,0.940204,1.014070,...,0.017924,0.034927,"[58.7, 57.87, 56.73, 57.22, 56.95, 57.68, 57.0...",4012,0.001055,-0.001482,0.002537,Comerica Inc (MI),Financials,STAY


As you can see, the table contains daily open and close data for SP500 stocks. The table also contains some features that have been generated for you using [navigation functions](https://cloud.google.com/bigquery/docs/reference/standard-sql/navigation_functions) and [analytic functions](https://cloud.google.com/bigquery/docs/reference/standard-sql/analytic-function-concepts). Let's dig into the schema a bit more. 

In [3]:
%%bigquery 
SELECT
    * EXCEPT(is_generated, generation_expression, is_stored, is_updatable)
FROM
    `cloud-training-prod-bucket.ml4f`.INFORMATION_SCHEMA.COLUMNS
WHERE
    table_name = "percent_change_sp500"

Downloading: 100%|██████████| 26/26 [00:01<00:00, 21.96rows/s]


,table_catalog,table_schema,table_name,column_name,ordinal_position,is_nullable,data_type,is_hidden,is_system_defined,is_partitioning_column,clustering_ordinal_position
0,cloud-training-prod-bucket,ml4f,percent_change_sp500,symbol,1,YES,STRING,NO,NO,NO,NaN
1,cloud-training-prod-bucket,ml4f,percent_change_sp500,Date,2,YES,DATE,NO,NO,NO,NaN
2,cloud-training-prod-bucket,ml4f,percent_change_sp500,Open,3,YES,FLOAT64,NO,NO,NO,NaN
3,cloud-training-prod-bucket,ml4f,percent_change_sp500,Close,4,YES,FLOAT64,NO,NO,NO,NaN
4,cloud-training-prod-bucket,ml4f,percent_change_sp500,tomorrow_close,5,YES,FLOAT64,NO,NO,NO,NaN
5,cloud-training-prod-bucket,ml4f,percent_change_sp500,tomo_close_m_close,6,YES,FLOAT64,NO,NO,NO,NaN
6,cloud-training-prod-bucket,ml4f,percent_change_sp500,close_MIN_prior_5_days,7,YES,FLOAT64,NO,NO,NO,NaN
7,cloud-training-prod-bucket,ml4f,percent_change_sp500,close_MIN_prior_20_days,8,YES,FLOAT64,NO,NO,NO,NaN
8,cloud-training-prod-bucket,ml4f,percent_change_sp500,close_MIN_prior_260_days,9,YES,FLOAT64,NO,NO,NO,NaN
9,cloud-training-prod-bucket,ml4f,percent_change_sp500,close_MAX_prior_5_days,10,YES,FLOAT64,NO,NO,NO,NaN


Most of the features, like `open` and `close` are pretty straightforward. The features generated using analytic functions, such as `close_MIN_prior_5_days` are best described using an example. Let's take the 6 most recent rows of data for IBM and reproduce the `close_MIN_prior_5_days` column. 

In [4]:
%%bigquery
SELECT 
    *
FROM
    `cloud-training-prod-bucket.ml4f.percent_change_sp500`
WHERE
    symbol = 'IBM'
ORDER BY 
    Date DESC
LIMIT 6

Downloading: 100%|██████████| 6/6 [00:01<00:00,  4.77rows/s]


,symbol,Date,Open,Close,tomorrow_close,tomo_close_m_close,close_MIN_prior_5_days,close_MIN_prior_20_days,close_MIN_prior_260_days,close_MAX_prior_5_days,...,close_STDDEV_prior_20_days,close_STDDEV_prior_260_days,close_values_prior_260,days_on_market,scaled_change,s_p_scaled_change,normalized_change,company,industry,direction
0,IBM,2013-02-01,204.65,205.18,NaN,NaN,0.989716,0.937323,0.879813,0.998977,...,0.025808,0.031267,"[180.52, 188.52, 189.98, 191.93, 191.73, 190.9...",12860,NaN,-0.011539,NaN,Intl Business Machines Corp,Information Technology,STAY
1,IBM,2013-01-31,203.32,203.07,205.18,2.11,1.002216,0.947063,0.888955,1.009356,...,0.025220,0.031925,"[181.07, 180.52, 188.52, 189.98, 191.93, 191.7...",12859,0.010391,0.010053,0.000338,Intl Business Machines Corp,Information Technology,STAY
2,IBM,2013-01-30,203.69,203.52,203.07,-0.45,1.001867,0.941185,0.884434,1.007125,...,0.024643,0.032221,"[180.0, 181.07, 180.52, 188.52, 189.98, 191.93...",12858,-0.002211,-0.002563,0.000352,Intl Business Machines Corp,Information Technology,STAY
3,IBM,2013-01-29,204.34,203.90,203.52,-0.38,0.961648,0.930996,0.878666,1.005248,...,0.023987,0.032552,"[179.16, 180.0, 181.07, 180.52, 188.52, 189.98...",12857,-0.001864,-0.003900,0.002036,Intl Business Machines Corp,Information Technology,STAY
4,IBM,2013-01-28,204.85,204.93,203.90,-1.03,0.948958,0.926316,0.874250,1.000195,...,0.021542,0.032677,"[180.55, 179.16, 180.0, 181.07, 180.52, 188.52...",12856,-0.005026,0.005106,-0.010132,Intl Business Machines Corp,Information Technology,DOWN
5,IBM,2013-01-25,204.45,204.97,204.93,-0.04,0.944772,0.926136,0.874079,0.998780,...,0.018510,0.032874,"[182.32, 180.55, 179.16, 180.0, 181.07, 180.52...",12855,-0.000195,-0.001850,0.001655,Intl Business Machines Corp,Information Technology,STAY


For `Date = 2013-02-01` how did we arrive at `close_MIN_prior_5_days = 0.989716`? The minimum close over the past five days was `203.07`. This is normalized by the current day's close of `205.18` to get `close_MIN_prior_5_days = 203.07 / 205.18 = 0.989716`. The other features utilizing analytic functions were generated in a similar way. Here are explanations for some of the other features:

* __scaled_change__: `tomo_close_m_close / close`
* __s_p_scaled_change__: This value is calculated the same way as `scaled_change` but for the S&P 500 index. 
* __normalized_change__: `scaled_change - s_p_scaled_change` The normalization using the S&P index fund helps ensure that the future price of a stock is not due to larger market effects. Normalization helps us isolate the factors contributing to the performance of a stock_market.
* __direction__: This is the target variable we're trying to predict. The logic for this variable is as follows: 

    ```sql
    CASE 
        WHEN normalized_change < -0.01 THEN 'DOWN'
        WHEN normalized_change > 0.01 THEN 'UP'
        ELSE 'STAY'
    END AS direction
    ```

## Create classification model for `direction`

In this example, your job is to create a classification model to predict the `direction` of each stock. Be creative! You can do this in any number of ways. For example, you can use BigQuery, Scikit-Learn, or AutoML. Feel free to add additional features, or use time series models.   



### Establish a Simple Benchmark

One way to assess the performance of a model is to compare it to a simple benchmark. We can do this by seeing what kind of accuracy we would get using the naive strategy of just predicting the majority class. Across the entire dataset, the majority class is 'STAY'. Using the following query we can see how this naive strategy would perform.

In [5]:
%%bigquery
WITH subset as (
    SELECT 
        Direction
    FROM
        `cloud-training-prod-bucket.ml4f.percent_change_sp500`
    WHERE
        tomorrow_close IS NOT NULL
)
SELECT 
    Direction,
    100.0 * COUNT(*) / (SELECT COUNT(*) FROM subset) as percentage
FROM
    subset
GROUP BY
    Direction

Downloading: 100%|██████████| 3/3 [00:01<00:00,  1.54rows/s]


,Direction,percentage
0,STAY,53.766049
1,UP,23.240681
2,DOWN,22.993271


So, the naive strategy of just guessing the majority class would have accuracy of around 54% across the entire dataset. See if you can improve on this. 

### Train Your Own Model

In [16]:
# TODO: Write code to build a model to predict Direction